In [9]:
from datetime import datetime
import pandas as pd
import numpy as np
import holidays
from typing import Optional

In [47]:
file_path = './data/'
TICKER = '^GSPC'
TIME_FRAME = '1d'

In [ ]:
# To use Christian's daily futures data, set USE_FUTURES_DAILY_DATA to True.  To use the Yahoo Finance daily data, set USE_FUTURES_DAILY_DATA to False.
USE_FUTURES_DAILY_DATA = False

# Turn gap on by setting USE_GAP to True.  Turn it off by setting USE_GAP to False.
USE_GAP = True

In [48]:
# Get the data
if USE_FUTURES_DAILY_DATA:
    df = pd.read_csv('./data/futures_d.csv')

else:
    df = pd.read_csv(f'./data/{TICKER}_{TIME_FRAME}.csv')
df.Date = pd.to_datetime(df.Date)
df['Date'] = df['Date'].dt.normalize()
df = df[(df['Date'] >= '2019-08-05')]
df.reset_index(drop=True, inplace=True)
us_holidays = holidays.US()
df['Holiday'] = df['Date'].apply(lambda x: 1 if x.strftime('%Y-%m-%d') in us_holidays else 0)

df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Holiday
0,0,2019-08-05,2898.070068,2898.070068,2822.120117,2844.739990,2844.739990,4542850000,0
1,1,2019-08-06,2861.179932,2884.399902,2847.419922,2881.770020,2881.770020,4156640000,0
2,2,2019-08-07,2858.649902,2892.169922,2825.709961,2883.979980,2883.979980,4512720000,0
3,3,2019-08-08,2896.209961,2938.719971,2894.469971,2938.090088,2938.090088,4118530000,0
4,4,2019-08-09,2930.510010,2935.750000,2900.149902,2918.649902,2918.649902,3356850000,0
...,...,...,...,...,...,...,...,...,...
1253,1253,2024-07-29,5476.549805,5487.740234,5444.439941,5463.540039,5463.540039,3379970000,0
1254,1254,2024-07-30,5478.729980,5489.459961,5401.700195,5436.439941,5436.439941,3777740000,0
1255,1255,2024-07-31,5505.589844,5551.509766,5493.750000,5522.299805,5522.299805,4546910000,0
1256,1256,2024-08-01,5537.839844,5566.160156,5410.419922,5446.680176,5446.680176,4703620000,0


In [49]:
df_small = pd.read_csv(f'./data/ESU24.CME_5m.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small = df_small[df_small['Date'] >= df['Date'][0]]
df_small.reset_index(drop=True, inplace=True)

df_small

,ESU24,Date,Open,High,Low,Last,Volume
0,ESU24,2019-08-05 00:00:00,3182.25,3183.75,3182.00,3183.25,1462
1,ESU24,2019-08-05 00:05:00,3183.25,3185.25,3181.75,3184.75,5295
2,ESU24,2019-08-05 00:10:00,3184.75,3186.00,3184.25,3185.75,3781
3,ESU24,2019-08-05 00:15:00,3185.75,3186.50,3184.75,3186.50,3833
4,ESU24,2019-08-05 00:20:00,3186.50,3187.50,3184.75,3185.75,3510
...,...,...,...,...,...,...,...
348156,ESU24,2024-08-08 12:45:00,5331.25,5348.50,5331.00,5347.25,21807
348157,ESU24,2024-08-08 12:50:00,5347.25,5347.75,5338.25,5342.00,20451
348158,ESU24,2024-08-08 12:55:00,5342.00,5347.50,5337.25,5341.25,32271
348159,ESU24,2024-08-08 13:00:00,5341.25,5352.50,5334.00,5347.25,93713


In [50]:
class Trade:
    def __init__(self, trade_type: str, open_price: float, sl: float, sg: float, n_lots: int) -> None:
        self.trade_type = trade_type
        self.open_price = open_price
        self.sl = sl
        self.sg = sg
        self.n_lots = n_lots

    def is_buy(self) -> bool:
        return self.trade_type == 'buy'

In [51]:
class SimulationResults:
    def __init__(self) -> None:
        self.points = 0
        self.n_wins = 0
        self.n_losses = 0
        self.win_streak = 0
        self.loss_streak = 0
        self.curr_win_streak = 0
        self.curr_loss_streak = 0
        self.n_buys = 0
        self.n_sells = 0
        self.n_ticks = 0
        self.dollars = 0

    def update(self, trade: Trade, trade_amount: float) -> None:
        self.n_buys += 1 if trade.is_buy() else 0
        self.n_sells += 1 if not trade.is_buy() else 0
        self.points += trade_amount
        n_ticks = trade_amount * 4
        dollars = n_ticks * 1.25 * trade.n_lots
        self.n_ticks += n_ticks
        self.dollars += dollars - 0.4
        self.n_wins += 1 if trade_amount > 0 else 0
        self.n_losses += 1 if trade_amount < 0 else 0
        self.curr_win_streak = (self.curr_win_streak + 1) if trade_amount > 0 else 0
        self.curr_loss_streak = (self.curr_loss_streak + 1) if trade_amount < 0 else 0
        self.win_streak = max(self.win_streak, self.curr_win_streak)
        self.loss_streak = max(self.loss_streak, self.curr_loss_streak)

    def __str__(self) -> str:
        return f'Points: {self.points}\nTicks won: {self.n_ticks}\nDollars won: {self.dollars}\nNum buys: {self.n_buys}\nNum sells: {self.n_sells}\nNum wins: {self.n_wins}\nNum losses: {self.n_losses}\nWin streak: {self.win_streak}\nLoss streak: {self.loss_streak}'

In [52]:
def run_simulation(num_points_diff: int, stop_gain_points: int, stop_loss_points: int, remove_holidays: bool, n_lots: int, can_buy: bool, can_sell: bool, gap: bool) -> SimulationResults:
    place_trade, buy, sim_results = False, False, SimulationResults()
    points_diff = num_points_diff
    sg_points, sl_points = stop_gain_points, stop_loss_points
    i = 1 if gap else 0

    def _iterate_through_trade(start_date: datetime, place_buy: bool) -> Optional[datetime]:
        # new_date = start_date.replace(hour=13, minute=30)
        new_date = start_date.replace(hour=6, minute=30) if gap else start_date.replace(hour=15, minute=0) 
        curr_df_small = df_small[df_small['Date'] >= new_date]
        curr_df_small.reset_index(drop=True, inplace=True)

        if len(curr_df_small) == 0:
            return None

        open_price = curr_df_small.loc[curr_df_small.index[0], 'Open']

        if place_buy:
            sl = open_price + 1 - sl_points
            sg = open_price + 1 + sg_points
            trade = Trade('buy', open_price, sl, sg, n_lots)

        else:
            sl = open_price - 1 + sl_points
            sg = open_price - 1 - sg_points
            trade = Trade('sell', open_price, sl, sg, n_lots)


        for j in range(len(curr_df_small)):
            curr_date, curr_low, curr_high = curr_df_small.loc[curr_df_small.index[j], ['Date', 'Low', 'High']]

            if trade.is_buy():
                if curr_low <= trade.sl:
                    trade_amount = trade.sl - trade.open_price
                    sim_results.update(trade, trade_amount)
                    return curr_date

                if curr_high >= trade.sg:
                    trade_amount = trade.sg - trade.open_price
                    sim_results.update(trade, trade_amount)
                    return curr_date

            else:
                if curr_high >= trade.sl:
                    trade_amount = trade.open_price - trade.sl
                    sim_results.update(trade, trade_amount)
                    return curr_date

                if curr_low <= trade.sg:
                    trade_amount = trade.open_price - trade.sg
                    sim_results.update(trade, trade_amount)
                    return curr_date
                
        return None

    while i < len(df):
        if not place_trade:
            date, curr_open, curr_close, is_holiday = df.loc[df.index[i], ['Date', 'Open', 'Close', 'Holiday']]
            if gap:
                curr_close = df.loc[df.index[i - 1], 'Close']
            not_a_holiday = not (remove_holidays and is_holiday)

            if can_buy and curr_close < curr_open and abs(curr_open - curr_close) >= points_diff and not_a_holiday:
                place_trade, buy = True, True

            elif can_sell and curr_close > curr_open and abs(curr_open - curr_close) >= points_diff and not_a_holiday:
                place_trade, buy = True, False

        if place_trade:
            trade_end_date = _iterate_through_trade(date, buy)
            place_trade, buy = False, False

            if trade_end_date is None:
                break

            else:
                i = int(df.loc[df['Date'] <= trade_end_date].index[-1] + 1)

        else:
            i += 1

    return sim_results

In [53]:
num_points = [20, 30, 50, 100]
sg_points = [20, 50, 100]
sl_points = [20, 50, 100]
remove_holidays_vals = [False]
can_buy_vals = [True, False]
can_sell_vals = [True, False]
all_combos = []

for n_points in num_points:
    for sg_point in sg_points:
        for sl_point in sl_points:
            for remove_holiday in remove_holidays_vals:
                for can_b in can_buy_vals:
                    for can_s in can_sell_vals:
                        if not can_b and not can_s:
                            continue
                        all_combos.append((n_points, sg_point, sl_point, remove_holiday, can_b, can_s))

best_num_points = None
best_sg_points = None
best_sl_points = None
best_remove_holidays_vals = None
best_can_buy_vals = None
best_can_sell_vals = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished, n_runs = 0, len(all_combos)
best_rewards = []
best_results = None

print(f'N runs: {n_runs}')

for n_points, sg_point, sl_point, remove_holiday, can_b, can_s in all_combos:
    results = run_simulation(num_points_diff=n_points, stop_gain_points=sg_point, stop_loss_points=sl_point, remove_holidays=remove_holiday, n_lots=1, can_buy=can_b, can_sell=can_s, gap=USE_GAP)
    profit = results.dollars
    runs_finished += 1

    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
    if min_item is None or profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': profit, 'n_points': n_points, 'sg_points': sg_point, 'sl_points': sl_point, 'remove_holiday': remove_holiday, 'can_buy': can_b, 'can_sell': can_s})

    if profit > best_reward:
        best_num_points = n_points
        best_sg_points = sg_point
        best_sl_points = sl_point
        best_remove_holidays_vals = remove_holiday
        best_can_buy_vals = can_b
        best_can_sell_vals = can_s
        best_results = results
        best_reward = profit


    print('Best profit so far: ' + str(best_reward) + '\n')

N runs: 108
Remaining runs: 107
Best profit so far: 1297.1999999999866

Remaining runs: 106
Best profit so far: 1297.1999999999866

Remaining runs: 105
Best profit so far: 1297.1999999999866

Remaining runs: 104
Best profit so far: 1297.1999999999866

Remaining runs: 103
Best profit so far: 1297.1999999999866

Remaining runs: 102
Best profit so far: 1297.1999999999866

Remaining runs: 101
Best profit so far: 1297.1999999999866

Remaining runs: 100
Best profit so far: 1297.1999999999866

Remaining runs: 99
Best profit so far: 2197.999999999992

Remaining runs: 98
Best profit so far: 2197.999999999992

Remaining runs: 97
Best profit so far: 2197.999999999992

Remaining runs: 96
Best profit so far: 2197.999999999992

Remaining runs: 95
Best profit so far: 2197.999999999992

Remaining runs: 94
Best profit so far: 2197.999999999992

Remaining runs: 93
Best profit so far: 2197.999999999992

Remaining runs: 92
Best profit so far: 2197.999999999992

Remaining runs: 91
Best profit so far: 2197.

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best distance points: ' + str(best_num_points))
print('Best sg points: ' + str(best_sg_points))
print('Best sl points: ' + str(best_sl_points))
print('Best remove holiday val: ' + str(best_remove_holidays_vals))
print('Best can buy val: ' + str(best_can_buy_vals))
print('Best can sell val: ' + str(best_can_sell_vals))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 13607.200000000039
Best distance points: 20
Best sg points: 100
Best sl points: 100
Best remove holiday val: False
Best can buy val: True
Best can sell val: False
-----------------------
Top results:
{'reward': 7012.600000000025, 'n_points': 20, 'sg_points': 20, 'sl_points': 50, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 6184.400000000014, 'n_points': 20, 'sg_points': 20, 'sl_points': 100, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 8953.60000000008, 'n_points': 20, 'sg_points': 50, 'sl_points': 50, 'remove_holiday': False, 'can_buy': True, 'can_sell': True}
{'reward': 5665.400000000007, 'n_points': 20, 'sg_points': 50, 'sl_points': 50, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 7041.200000000028, 'n_points': 20, 'sg_points': 50, 'sl_points': 100, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 5099.4000000000215, 'n_points':

In [11]:
print('Best results:')
print(best_results)

Best results:
Points: 2732.0
Ticks won: 10928.0
Dollars won: 13607.200000000039
Num buys: 132
Num sells: 0
Num wins: 79
Num losses: 53
Win streak: 8
Loss streak: 7
